In [1]:
import os
import ants

In [2]:
# anat_path = input('Enter path for Anatomy Scan:\n')
# bold_stim_path = input('Enter path for BOLD Stimuli Scan:\n')
# bold_rest_path = input('Enter path for BOLD Rest Path')
# mni_template_path = input('Enter path for MNI Template:\n')
# mni_mask_img = input('Enter path for MNI Mask:\n')

In [3]:
# For testing purposes so you don't have to keep typing stuff in
parent_dir = (lambda: os.path.dirname(os.path.dirname(os.getcwd())))
anat_path = os.path.join(parent_dir(), "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_path = os.path.join(parent_dir(), "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_path = os.path.join(parent_dir(), "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_template_path =  os.path.join(parent_dir(), "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_path = os.path.join(parent_dir(), "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")

In [4]:
#  quick debug
print(anat_path, bold_stim_path, bold_rest_path, mni_template_path, mni_mask_path)

/Users/joachimpfefferkorn/repos/neurovolume/media/sub-01/anat/sub-01_T1w.nii.gz /Users/joachimpfefferkorn/repos/neurovolume/media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz /Users/joachimpfefferkorn/repos/neurovolume/media/sub-01/func/sub-01_task-rest_bold.nii.gz /Users/joachimpfefferkorn/repos/neurovolume/templates/mni_icbm152_t1_tal_nlin_sym_09a.nii /Users/joachimpfefferkorn/repos/neurovolume/templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii


In [5]:
bold_stim_img = ants.image_read(bold_stim_path)
# bold_rest_img = ants.image_read(bold_rest_path)
ants_img_string = ants.image_read(anat_path)
# mni_template_img = ants.image_read(mni_template_path)
# mni_mask_img = ants.image_read(mni_mask_path)

In [6]:
from functions import *

Lets get some metadata

In [138]:
import re

In [195]:
def parse_metadata(ants_img_string):
    info = str(ants_img_string).split('\n')
    metadata = {}
    for idx, entry in enumerate(info):
        strips = entry.strip().split(":")
        if len(strips) == 1 and idx==0:
            metadata["Header"] = strips[0].strip()
        elif len(strips) == 2:
            name, value = strips
            metadata[name.strip()] = value
        else:
            continue
    return metadata
metadata = parse_metadata(anat_img)
print(metadata)

{'Header': 'ANTsImage (LPI)', 'Pixel Type': ' float (float32)', 'Components': ' 1', 'Dimensions': ' (512, 512, 296)', 'Spacing': ' (0.4785, 0.4785, 0.5)', 'Origin': ' (119.989, 104.52, -84.2457)', 'Direction': ' [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]'}


In [27]:
parse_metadata(bold_stim_img)

ANTsImage
-
	 Pixel Type : float (float32)
-
	 Components : 1
-
	 Dimensions : (64, 64, 35, 185)
-
	 Spacing    : (4.0, 4.0, 4.0, 2.0)
-
	 Origin     : (-127.953, 108.933, -74.8393, 0.0)
-
	 Direction  : [ 1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
-

-


🤔

Should these be normalized at this stage? maybe not? maybe do that in blender with numpy?

In [8]:
t1_vol = create_volume(ants_img_string.numpy())
bold_stim_vol = create_volume(bold_stim_img.numpy())

In [9]:
bold_rest_vol = create_volume(bold_rest_img.numpy())

In [10]:
mask = generate_brain_mask(ants_img_string, mni_template_img, mni_mask_img)
mask_vol = create_volume(mask.numpy())

Creating brain mask


# Quick Viewer Sanity checks

In [11]:
explore_3D_vol(t1_vol[:,:,:], dim='z')

interactive(children=(IntSlider(value=147, description='slice', max=295), Output()), _dom_classes=('widget-int…

In [ ]:
explore_3D_vol(mask_vol, dim='z')

In [ ]:
explore_3D_vol(bold_stim_vol[30,:,:,:], dim='z')

In [ ]:
explore_3D_vol(bold_rest_vol[30,:,:,:], dim='z')

# Back to our Regularly Scheduled Programming

In [72]:
def get_name(var):
    """
    Returns the name of a variable
    """
    for name, value in globals().items():
        if value is var:
            return name

In [ ]:
print(get_name(t1_vol))

In [77]:
def save_npy(vol: np.ndarray, output_folder: str):
    with open(f'{output_folder}/{get_name(vol)}.npy', 'wb') as f:
        np.save(f, np.array(vol))

In [78]:
output_folder = "/Users/joachimpfefferkorn/repos/neurovolume/output"

In [79]:
save_npy(t1_vol, output_folder)

# View Exports dummy check

In [2]:
import numpy as np

In [3]:
anatomy_check =  np.load("/Users/joachimpfefferkorn/repos/neurovolume/output/anat.npy")

In [11]:
explore_3D_vol(anatomy_check[:,:,:], dim='z')

interactive(children=(IntSlider(value=147, description='slice', max=295), Output()), _dom_classes=('widget-int…